In [99]:
import pickle

In [26]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [27]:
from geopy import distance
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import r2_score as R2
from sklearn.metrics import root_mean_squared_error as RMSE, roc_auc_score as auc
from sklearn.feature_selection import RFECV
import sklearn_genetic
from sklearn.model_selection import GridSearchCV

In [28]:
from sklearn.neural_network import MLPClassifier as MLP

In [29]:
from imblearn.over_sampling import SMOTE

In [57]:
df = pd.read_csv("C4K.csv")
zip_df = pd.read_csv("DemographicData.csv")

In [58]:
zip_shapes = gpd.read_file("zipShapes/oklahoma-zip-code-boundaries.shp")
float_items = []
for item in zip_shapes["zcta5ce00"]:
    try: 
        float_items.append(float(item))
    except:
        pass
non_code= [element for element in float_items if element not in np.array(df["ZIP CODE"])] 

In [59]:
df = df.drop("Unnamed: 0", axis = 1)

In [33]:
date_array = np.concat([np.full(len(non_code), 2018),np.full(len(non_code), 2019),np.full(len(non_code), 2020),np.full(len(non_code), 2021), np.full(len(non_code), 2022),np.full(len(non_code), 2023)], axis = 0)

In [34]:
filled_df = pd.DataFrame(list(zip(np.tile(non_code,6),np.zeros(len(non_code)*6), date_array)),columns = ["ZIP CODE", "Given", "DATE"])

In [35]:
filled_df

,ZIP CODE,Given,DATE
0,74761.0,0.0,2018
1,74738.0,0.0,2018
2,74764.0,0.0,2018
3,74766.0,0.0,2018
4,73548.0,0.0,2018
...,...,...,...
2251,74842.0,0.0,2023
2252,73661.0,0.0,2023
2253,73664.0,0.0,2023
2254,73841.0,0.0,2023


In [36]:
df = pd.concat([filled_df, df], axis = 0)

In [60]:
df

,DATE,ZIP CODE,Given
0,2018,70311.0,12.0
1,2018,73002.0,5.0
2,2018,73003.0,4.0
3,2018,73005.0,85.0
4,2018,73008.0,271.0
...,...,...,...
859,2023,74953.0,200.0
860,2023,74955.0,200.0
861,2023,74960.0,250.0
862,2023,78120.0,8.0


In [61]:
test_df =df.groupby(["DATE", "ZIP CODE"])["Given"].apply(np.sum, axis = 0).reset_index()
test_df.set_index("ZIP CODE").loc["74953"]

KeyError: '74953'

In [62]:
df = df.set_index("ZIP CODE")

In [63]:
df.index = np.array(df.index).astype('U5').astype(str)
df.index.name = "ZIP CODE"

In [64]:
zip_df = zip_df.rename({"Your Entry": "ZIP CODE"}, axis = 1).set_index("ZIP CODE")

In [65]:
zip_df.index = np.array(zip_df.index).astype(str) 

In [66]:
full_df = pd.merge(df, zip_df, right_index=True, left_index=True)

In [67]:
full_df = full_df[(full_df["Population"] != 0) & (full_df["ACS Race Indian"] != 0)] 

In [68]:
full_df = full_df[["Population", "Business Annual Payroll", "Households", "Income Per Household", "Latitude", "Longitude", "Asian Population", "Black or African American Population", "American Indian or Alaskan Native Population", "Native Hawaiian or other Pacific Islander Population", "White Population", "DATE", "Given"]]

In [69]:
lats = full_df["Latitude"].values
longs = full_df["Longitude"].values
coords_2 = (35.55019107302975, -97.5261067901524)

In [70]:
miles = []

In [71]:
for i in range(len(lats)):
    coords_1 = (lats[i], longs[i])
    miles.append(distance.geodesic(coords_1, coords_2).miles)

In [72]:
full_df["Distance"] = miles

In [73]:
target_column = full_df.pop("Given")

In [74]:
full_df["DATE"]= full_df["DATE"] - 2017

In [75]:
# feature ideas
# Buiness Annual Payroll / Population
# Buisness Annual Payroll / Number of Employees

In [76]:
full_df["Payroll per Population"] = full_df['Business Annual Payroll']/full_df['Population']
full_df["Payroll per Employee"] = full_df['Business Annual Payroll']/full_df['Number of Employees']

KeyError: 'Number of Employees'

In [77]:
def convertFloat(number):
    #75
    if number > 25:
        return 1
    else:
        return 0

In [78]:
target_column2 = pd.Series(list(map(convertFloat, target_column.values)), index = target_column.index)

In [79]:
target_column2

ZIP CODE
73002    0
73003    0
73005    1
73008    1
73011    0
        ..
74884    0
74953    1
74955    1
74960    1
73132    0
Length: 786, dtype: int64

In [80]:
target_column.index

Index(['73002', '73003', '73005', '73008', '73011', '73012', '73013', '73015',
       '73016', '73017',
       ...
       '74868', '74871', '74872', '74873', '74881', '74884', '74953', '74955',
       '74960', '73132'],
      dtype='object', name='ZIP CODE', length=786)

In [81]:
full_df = full_df.select_dtypes(include="number")

In [82]:
full_df

,Population,Business Annual Payroll,Households,Income Per Household,Latitude,Longitude,Asian Population,Black or African American Population,American Indian or Alaskan Native Population,Native Hawaiian or other Pacific Islander Population,White Population,DATE,Distance,Payroll per Population
ZIP CODE,,,,,,,,,,,,,,
73002,967,4350,377.0,58688.0,34.980390,-97.757286,2,1,61,0,786,1,41.398688,4.498449
73003,22648,205816,9154.0,66384.0,35.667505,-97.496047,675,2098,628,36,15669,1,8.263051,9.087602
73005,8072,88508,3003.0,44517.0,35.123680,-98.232820,33,308,3431,3,3020,1,49.581571,10.964817
73008,21592,188432,8215.0,52122.0,35.518612,-97.645090,309,1801,716,24,12927,1,7.049606,8.726936
73011,377,2871,148.0,50357.0,34.870020,-97.730538,0,0,25,0,320,1,48.294872,7.615385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74884,5509,30620,2073.0,39696.0,35.225782,-96.541022,17,574,1190,0,2930,6,59.940890,5.558178
74953,12113,123522,4579.0,46810.0,35.084916,-94.579552,93,118,1613,17,8138,6,169.546083,10.197474
74955,14340,112076,5592.0,44080.0,35.525745,-94.747424,108,152,3602,2,8095,6,156.582903,7.815621


In [83]:
X_train, X_test, y_train, y_test = train_test_split(full_df,target_column2, random_state=1)

In [84]:
X_train.dropna(axis = 1, inplace=True)

In [85]:
sm = SMOTE(random_state=12)
X_train, y_train = sm.fit_resample(X_train, y_train)

In [86]:
corr = X_train.corrwith(y_train, numeric_only=True)
corr_features = corr.sort_values(ascending=False).index[:30]

In [87]:
corr

Population                                              0.189766
Business Annual Payroll                                 0.232582
Households                                              0.198665
Income Per Household                                   -0.164266
Latitude                                                0.000174
Longitude                                               0.054629
Asian Population                                        0.155692
Black or African American Population                    0.252621
American Indian or Alaskan Native Population            0.160411
Native Hawaiian or other Pacific Islander Population    0.010323
White Population                                        0.123000
DATE                                                    0.135822
Distance                                                0.016566
Payroll per Population                                  0.084462
dtype: float64

In [88]:
corr_features = corr[corr > .1].index

In [89]:
corr_features

Index(['Population', 'Business Annual Payroll', 'Households',
       'Asian Population', 'Black or African American Population',
       'American Indian or Alaskan Native Population', 'White Population',
       'DATE'],
      dtype='object')

In [1286]:
# feature ideas
# Buiness Annual Payroll / Population
# Buisness Annual Payroll / Number of Employees

In [90]:
rfecv = RFECV(RandomForestClassifier(), cv= 5, step = 1)
rfecv.fit(X_train[corr_features], y_train)

KeyboardInterrupt: 

In [980]:
rfecv_features = rfecv.get_feature_names_out(corr_features)

In [981]:
rfecv_features

array(['Population', 'Business Annual Payroll', 'Households',
       'Asian Population', 'Black or African American Population',
       'American Indian or Alaskan Native Population', 'White Population'],
      dtype=object)

In [985]:
auc(rfecv.predict(X_test[rfecv_features]), y_test)

np.float64(0.6094735201745343)

In [992]:
np.array(y_test)

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,

In [991]:
rfecv.predict(X_test[rfecv_features])

array([0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,

In [798]:
hand_selected = ['Population', 'Business Annual Payroll', 'Households', 'Number of Businesses', 'Number of Employees', 'Black or African American Population', 'American Indian or Alaskan Native Population', 'White Population']

In [359]:
ga_fs = sklearn_genetic.GAFeatureSelectionCV(RandomForestClassifier(), cv = 3, scoring="roc_auc", generations=30, population_size= 10)
ga_fs.fit(X_train[corr_features], y_train)

C:\Users\kapat\Desktop\Capstone\venv\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\kapat\Desktop\Capstone\venv\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	10    	0.937018	0.00185943 	0.939737   	0.933994   
1  	20    	0.93789 	0.00115291 	0.939391   	0.936264   
2  	20    	0.938067	0.00107609 	0.939391   	0.936264   
3  	20    	0.93873 	0.00192969 	0.941047   	0.935192   
4  	20    	0.939431	0.00117784 	0.941047   	0.937916   
5  	20    	0.938127	0.00260234 	0.941047   	0.93266    
6  	20    	0.938152	0.00226169 	0.941047   	0.934644   
7  	20    	0.939675	0.00162762 	0.941047   	0.935403   
8  	20    	0.940543	0.000500901	0.941047   	0.939691   
9  	20    	0.939478	0.00263893 	0.941047   	0.933745   
10 	20    	0.939785	0.00176889 	0.941047   	0.934895   
11 	20    	0.940174	0.0010609  	0.941047   	0.93771    
12 	20    	0.939789	0.00192485 	0.941411   	0.935586   
13 	20    	0.939607	0.00255958 	0.941411   	0.932535   
14 	20    	0.939592	0.00269784 	0.941411   	0.933265   
15 	20    	0.937875	0.0028128  	0.941411   	0.933853   
16 	20    	0.938343	0.00225545 	0.941411   	0.93

GAFeatureSelectionCV(estimator=RandomForestClassifier(), generations=30,
                     population_size=10, scoring='roc_auc')

In [362]:
corr_features

Index(['ACS Race Black', 'Black or African American Population',
       'ACS Race Two Or More', 'Households', 'ACS Households Total',
       'Delivery Total', 'Delivery Residential', 'Female Population',
       'Population', 'Male Population', 'ACS Population Female',
       'ACS Population', 'Population Estimate', 'ACS Population Male',
       'Delivery Business', 'ACS Race Non-Hispanic', 'Number of Businesses',
       'ACS Race Hispanic', 'Number of Employees', 'ACS Race Other',
       'ACS Race White', 'White Population', 'Other Population',
       'Asian Population', 'Business Annual Payroll',
       'Business First Quarter Payroll', 'ACS Race Indian', 'ACS Race Asian',
       'CBSA Population', 'American Indian or Alaskan Native Population'],
      dtype='object')

In [371]:
ga_fs.get_feature_names_out(corr_features)

array(['ACS Race Black', 'Black or African American Population',
       'ACS Race Two Or More', 'ACS Households Total',
       'Delivery Residential', 'Population', 'ACS Population Male',
       'ACS Race Non-Hispanic', 'Number of Businesses',
       'ACS Race Hispanic', 'ACS Race Other', 'ACS Race White',
       'Asian Population', 'Business Annual Payroll',
       'Business First Quarter Payroll', 'ACS Race Indian',
       'ACS Race Asian', 'CBSA Population',
       'American Indian or Alaskan Native Population'], dtype=object)

In [671]:
model = GradientBoostingClassifier()
model.fit(X_train[corr_features], y_train)
R2(y_test, model.predict(X_test[corr_features]))

-0.12387543252595168

In [27]:
y_test

ZIP CODE
79111     33.0
74747      0.0
73772      0.0
74745    183.0
73835      0.0
         ...  
74731      0.0
74839      0.0
74643      0.0
73054    251.0
73756      0.0
Name: Given, Length: 952, dtype: float64

In [53]:
model.predict(X_test[rfecv_features])

array([ 1.91869475e+02,  3.50367132e+01,  6.87018440e+00,  9.32257722e+01,
        8.91353249e+01,  2.15967945e+03,  5.69646486e+02, -9.71487376e+00,
        3.89248443e+01, -7.32457954e+01,  1.96706257e+02,  2.33799739e+02,
        1.54238738e+02,  2.63708734e+02,  2.20126828e+02,  7.25247181e+01,
        2.49371248e+02,  2.53700075e+02,  1.87080219e+02,  7.87447881e+01,
        1.18077756e+02,  1.22712786e+02, -4.10691159e+01,  1.43864874e+02,
        3.21686886e+02,  1.40535174e+02,  1.57839365e+02,  2.27800345e+02,
        3.12925788e+02,  2.07100896e+02,  2.45168753e+02, -1.23847009e+01,
        2.08268853e+02,  4.86136373e+01,  2.37104944e+02,  7.18715795e+01,
        1.96280128e+02,  3.72549247e+02,  8.11071580e+01,  1.16189602e+02,
        1.30496774e+01,  1.32623181e+02,  2.80377142e+02,  2.19428235e+02,
        2.39384516e+02,  7.01977663e+01,  4.08704654e+01,  1.62188531e+02,
        4.18926865e+01,  6.78289565e+01,  2.52333981e+02,  1.04110314e+02,
        1.25865362e+02,  

In [821]:
neural_net = MLP(random_state=2, activation="relu").fit(X_train[hand_selected], y_train)

In [823]:
neural_net.score(X_test[hand_selected], y_test)

0.33613445378151263

In [824]:
auc(neural_net.predict(X_test[hand_selected]), y_test)

np.float64(0.5395555555555556)

In [1303]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.1, 0.01, 0.001],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 3, 4]
}
search = GridSearchCV(GradientBoostingClassifier(), param_grid, cv = 5)
search.fit(X_train[corr_features], y_train)

GridSearchCV(cv=5, estimator=GradientBoostingClassifier(),
             param_grid={'learning_rate': [0.1, 0.01, 0.001],
                         'max_depth': [3, 4, 5], 'min_samples_split': [2, 3, 4],
                         'n_estimators': [100, 200, 300]})

In [1305]:
search.best_params_

{'learning_rate': 0.1,
 'max_depth': 5,
 'min_samples_split': 2,
 'n_estimators': 200}

In [1359]:
corr_features

Index(['Population', 'Business Annual Payroll', 'Households',
       'Asian Population', 'Black or African American Population',
       'American Indian or Alaskan Native Population', 'White Population',
       'DATE'],
      dtype='object')

In [91]:
model = GradientBoostingClassifier(random_state=2)
model.fit(X_train[corr_features], y_train)
auc(model.predict(X_test[corr_features]), y_test)

np.float64(0.7314685314685315)

In [92]:
model.feature_importances_

array([0.04321108, 0.14210829, 0.07223426, 0.10695162, 0.24560952,
       0.11850281, 0.13407051, 0.1373119 ])

In [93]:
corr_features

Index(['Population', 'Business Annual Payroll', 'Households',
       'Asian Population', 'Black or African American Population',
       'American Indian or Alaskan Native Population', 'White Population',
       'DATE'],
      dtype='object')

In [94]:
len(np.atleast_1d(y_test).nonzero()[0])

149

In [95]:
len(np.atleast_1d(model.predict(X_test[corr_features])).nonzero()[0])

132

In [96]:
len(np.intersect1d(np.atleast_1d(y_test).nonzero(), np.atleast_1d(model.predict(X_test[corr_features])).nonzero()))

120

In [97]:
len(np.atleast_1d(y_test).nonzero()[0])

149

In [98]:
120/149

0.8053691275167785

In [103]:
with open('final_model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [104]:
with open('final_model.pkl', 'rb') as file:
    model = pickle.load(file)

In [105]:
model.fit(X_train[corr_features], y_train)
auc(model.predict(X_test[corr_features]), y_test)

np.float64(0.7314685314685315)